In [1]:
# tratamiento de datos local
import pandas as pd
import os
import re

#extraccion de variables de servicios externos
import socket
import requests
from english_words import english_words_lower_alpha_set
from bs4 import BeautifulSoup
from urllib.parse import urlencode

# utilities (loading bar)
from tqdm.notebook import tqdm
import time

In [2]:
def preProcessing(link):
    protocol = re.findall('://',link)
    slash_final = re.findall('/$', link)
    
    # añadir :// al principio si no tiene protocolo
    if len(protocol) == 0:
        link = '://' + link   
        
    # si al final no tiene / añadirlo
    if len(slash_final) == 0:
        link = link + '/'
    link_arr = re.findall('//[\w%]+.[\w\W]+?/', link)[0].replace('/','').split('.')
    
    # si no tiene subdominio añadirle www.
    if len(link_arr) < 3:
        link = link.replace('://','://www.')
    
    return link

def cleanArrayLink(link):
    # obtener un array con los componentes de la url antes del / final   
    link_array = re.findall('[\w]*://[\w\W]+?/', link)[0].replace('://','://.').split('.')
    
    # si no tiene tld añadirle .com
    if len(link_array) < 4:
        link_array[2] = link_array[2].replace('/','')
        link_array.append("com/")
        
    if len(link_array) > 0:
        return link_array
    else:
        return None
    
def getIP(link):
    try:
        IP_address = socket.gethostbyname(link)
    except:
        IP_address = None
    return IP_address

def getHostName(ip_address):
    try:
        hostname = socket.gethostbyaddr(ip_address)[0]
    except:
        hostname = None
    return hostname

def normalizarAcentos(palabra): # palabra se encuentra como tipo: byte
    a,b = 'áéíóúüñÁÉÍÓÚÜÑ','aeiouunAEIOUUN'
    trans = str.maketrans(a,b)
    
    return str(palabra,'utf-8').translate(trans)

In [1]:
def armarDB(links_df, nombre_archivo): #tiene que ser dataframe
    #if paso == 2:
    links_df.columns = ['link']

    print("# links antes de quitar duplicados: ", len(links_df))
    links_df = links_df.drop_duplicates()
    print("# links despues de quitar duplicados: ", len(links_df))
    links_sr =  links_df.squeeze() # pandas serie

    links_sr = links_sr.apply(preProcessing)
    split_arr_df = links_sr.apply(lambda x: cleanArrayLink(x))

    #attr: subominio, tld, protocolo
    subdomains = split_arr_df.apply(lambda x: x[1])
    top_level = split_arr_df.apply(lambda x: x[3].replace('/',''))
    protocol = split_arr_df.apply(lambda x: None if x[0] == '://' else x[0])
    norm_links = split_arr_df.apply(lambda x: '.'.join(x[1:]).replace('/',''))

    links_df['subdominio'] = subdomains
    links_df['top_level_domain'] = top_level
    links_df['protocolo'] = protocol

    patrones = [r'\.',"-","_","=","@","&","!",r"\+",r"\*","#",r"\$","%",r"\?","\d"]
    links_df["num ."] = links_sr.apply(lambda x: len(re.findall(patrones[0],x)))
    links_df["num -"] = links_sr.apply(lambda x: len(re.findall(patrones[1],x)))
    links_df["num _"] = links_sr.apply(lambda x: len(re.findall(patrones[2],x)))
    links_df["num ="] = links_sr.apply(lambda x: len(re.findall(patrones[3],x)))
    links_df["num @"] = links_sr.apply(lambda x: len(re.findall(patrones[4],x)))
    links_df["num &"] = links_sr.apply(lambda x: len(re.findall(patrones[5],x)))
    links_df["num !"] = links_sr.apply(lambda x: len(re.findall(patrones[6],x)))
    links_df["num +"] = links_sr.apply(lambda x: len(re.findall(patrones[7],x)))
    links_df["num *"] = links_sr.apply(lambda x: len(re.findall(patrones[8],x)))
    links_df["num #"] = links_sr.apply(lambda x: len(re.findall(patrones[9],x)))
    links_df["num $"] = links_sr.apply(lambda x: len(re.findall(patrones[10],x)))
    links_df["num %"] = links_sr.apply(lambda x: len(re.findall(patrones[11],x)))
    links_df["num ?"] = links_sr.apply(lambda x: len(re.findall(patrones[12],x)))
    links_df["num digitos"] = links_sr.apply(lambda x: len(re.findall(patrones[13],x)))

    links_df.to_csv(nombre_archivo + "_1.csv", index = False)
    print("el archivo se guardó como " + nombre_archivo + "_1.csv")
    
    # REVISAR PALBRAS
    print("buscando palabras en ingles...")
    links_df["num eng"] = links_sr.apply(lambda x:sum([1 if w in WORDS else 0 for w in re.findall("[\w]+",x)]))
    print("buscando palabras en español...")
    links_df["num esp"] = links_sr.apply(lambda x:sum([1 if w in PALABRAS else 0 for w in re.findall("[\w]+",x)]))
    print("buscando palabras fuera del diccionario...")
    links_df["num no dic"] = links_sr.apply(lambda x: len(re.findall("[\w]+",x)))-links_df["num eng"]-links_df["num esp"]
    
    links_df.to_csv(nombre_archivo + "_2.csv", index = False)
    print("el archivo se guardó como " + nombre_archivo + "_2.csv")
    
    return links_df, norm_links, nombre_archivo

def armarIpDF(links_df, norm_links, nombre_archivo):
    print("obteniendo las direcciones ip... (esto puede tardar horas)")
    ip_addresses = [getIP(address) for address in tqdm(norm_links)]
    links_df['ip_address'] = ip_addresses

    links_df.to_csv(nombre_archivo + "_3.csv", index = False)
    print("el archivo se guardó como " + nombre_archivo + "_3.csv")
    return links_df, nombre_archivo

# def armarHostnameDF(links_df, nombre_archivo):
#     print("obteniendo los hostnames... (esto puede tardar horas)")
#     hostnames = [getHostName(ip_address) for ip_address in tqdm(links_df["ip_address"])]
#     links_df['hostname'] = hostnames

#     links_df.to_csv(nombre_archivo + "_4.csv", index = False)
#     print("el archivo se guardó como " + nombre_archivo + "_4.csv")

#     return links_df, nombre_archivo

In [4]:
### VARIABLES ADICIONALES ###

# palabras en español
word_site = "https://raw.githubusercontent.com/JorgeDuenasLerin/diccionario-espanol-txt/master/0_palabras_todas.txt"
response = requests.get(word_site)

PALABRAS = response.content.splitlines()
PALABRAS = [normalizarAcentos(w) for w in PALABRAS]

# palabras en ingles
WORDS = english_words_lower_alpha_set 

In [7]:
data_ruta = r'D:\Documentos\UP\2021\2021-2\trabajo final\extract variables\data'
phishing_db = os.listdir(data_ruta)
df_ebbu = pd.read_json('data/data_legitimate_36400.json')
df_iscx = pd.read_csv('data/Benign_list_big_final.csv',header = None)
legit_df = pd.concat([df_ebbu, df_iscx]).reset_index(drop=True).drop_duplicates()
#df_phtk = pd.read_csv('data/phishtank2_ph.txt', sep = '\t', header = None) #ya limpie los ...
#df_phtk 
legit_df

,0
0,http://www.conceptdraw.com/How-To-Guide/Local-...
1,https://www.edrawsoft.com/Local-Area-Network.php
2,http://www.webopedia.com/TERM/L/local_area_net...
3,https://www.acsac.org/secshelf/book001/16.pdf
4,http://www.diffen.com/difference/LAN_vs_WAN
...,...
71773,https://lastpass.com/signup2.php?ac=1&from_uri...
71774,https://lastpass.com/signup2.php?ac=1&from_uri...
71775,https://lastpass.com/signup2.php?ac=1&from_uri...
71776,https://lastpass.com/signup2.php?ac=1&from_uri...


In [8]:
# WORK PIPE
links_df, norm_links, nombre_archivo = armarDB(legit_df, "ligit_df")
links_df, nombre_archivo = armarIpDF(links_df, norm_links, nombre_archivo)
#links_df, nombre_archivo = armarHostnameDF(links_df, nombre_archivo)

# links antes de quitar duplicados:  65288
# links despues de quitar duplicados:  65288
el archivo se guardó como ligit_df_1.csv
buscando palabras en ingles...
buscando palabras en español...
el archivo se guardó como ligit_df_2.csv
obteniendo las direcciones ip... (esto puede tardar horas)



el archivo se guardó como ligit_df_3.csv
obteniendo los hostnames... (esto puede tardar horas)


KeyboardInterrupt: 